<a href="https://colab.research.google.com/github/emmt1998/ML-para-Negocios/blob/main/workshop_02/Workshop_N02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop N02: Recomendador parte 1
Dictado por Sergio Valenzuela / sergio@tecnopolis.ai / serginho61@gmail.com

Redactado por Efrain Magaña / emmt1998@gmail.com

Puedes copiar este Jupyter Notebook a tu drive llendo a Archivo-> Guardar una Copia en Drive


# Carga de Dataset
En este workshop usaremos los datos en `ml-latest-small.zip` de https://grouplens.org/datasets/movielens/latest/ el cual corresponde a 100mil ratings y 3600 etiquetas aplicadas a 9000 peliculas por 600 usuarios.

En especifico usaremos los archivos `movies.csv` y `ratings.csv` que estan dentro del zip.

Hay dos metodos para cargar los archivos al ambiente de Google Collab, uno es manual, llendo al icono de carpeta en el costado izquierdo y luego presionar en el icono de un archivo con una flecha hacia arriba, tambien se pueden arrastrar los archivos. 

El otro metodo es ejecutar el siguiente codigo, este copiara el repositorio del curso en el ambiente y por ende copiara los archivos en la carpeta `ML-para-Negocios/workshop_02/ml-latest-small/` en donde ya estan descomprimidos `movies.csv` y `ratings.csv`.

In [2]:
%cd /content/
!rm -r /content/ML-para-Negocios #remueve el repositorio si es que ya se habia cargado
!git clone https://github.com/emmt1998/ML-para-Negocios.git #copia el repositorio

/content
rm: cannot remove '/content/ML-para-Negocios': No such file or directory
Cloning into 'ML-para-Negocios'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 71 (delta 22), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (71/71), done.


Si revisas en el icono de la carpeta encontraras que se cargo el repositorio al ambiente.

# Libreria Pandas
En esta ocacion usaremos la libreria pandas para trabajar con los archivos. 

En la siguiente pagina encontraras mas informacion sobre la libreria:

* https://pandas.pydata.org/docs/getting_started/index.html

Ejecutmos la siguiente celda para importar la libreria al ambiente.


In [3]:
import pandas as pd #notamos que le asignamos el nombre de pd

# Carga de datos
Primero cargamos los datos a variables para trabajar con ellos.

In [4]:
# Primero seteamos las direcciones en donde estan los archivos a usar
dir_archivos = "/content/ML-para-Negocios/workshop_02/ml-latest-small/"
dir_movies = dir_archivos +"movies.csv"
dir_ratings = dir_archivos + "ratings.csv"

# usamos el metodo de pandas para leer los csv
# primero con las peliculas
peliculas = pd.read_csv(dir_movies)
# aprovechamos de cambiarle los nobmres a las columnas
peliculas.columns = ["peliculaId", "titulo", "generos"]
# y de setear la columna de peliculasId como indice
peliculas = peliculas.set_index("peliculaId")

# ahora con los ratings
notas = pd.read_csv(dir_ratings)
# aprovechamos de cambiarle los nobmres a las columnas
notas.columns = ["usuarioId", "peliculaId", "nota", "momento"]

Podemos invocar a la variable para que se muestren los datos.

Primero invocamos el dataframe `peliculas`, en donde se pueden ver las columnas "peliculaId", "titulo" y "generos". Siendo la columna de "peliculaId" el indice de los datos.

In [6]:
peliculas   #tambien puedes usar peliculas.head(n) con n el numero de datos que quieres ver

,titulo,generos
peliculaId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
193585,Flint (2017),Drama


Invocamos al dataframe `notas`, notamos que el indice en este caso no corresponde a "peliculaId" y tampoco nos conviene que sea, ya que distintos usuarios le han puesto nota a una misma pelicula, por lo que si pusieramos como indice a las peliculas, tendriamos que se repiten varias veces.

In [7]:
notas

,usuarioId,peliculaId,nota,momento
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


Tambien podemos pedir una descripcion de los datos.

In [9]:
notas.describe()

,usuarioId,peliculaId,nota,momento
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


En esto ultimo podemos notar que la nota promedio es un 3.5 y que solo un 25% de los ratings corresponde a una nota menor a tres, podemos ignorar los resultados de "usuarioId" y "peliculaId" ya que son identificadores y a "momento" puesto que no lo usaremos.

# 1era Heurística de Recomendacion: Ranking de popularidad
Ahora, ¿Como recomiendo peliculas?, si no conozco al usuario lo mas facil es recomendar lo que es mas popular, pero si se habran dado cuenta, los datos que estamos usando no cuentan con un total de visitas. Pero sabemos las peliculas a las cuales cada usuario le dejo una calificacion, con esto podemos contar cuantas calificaciones recibio cada pelicula. Y asumiremos que entre mas calificaciones mas popular es una pelicula. Esto ultimo puede ser completamente falso, pero es una asuncion que nos lleva a una solucion.

In [11]:
# contamos cuantas notas le dejaron a cada pelicula
total_de_votos = notas["peliculaId"].value_counts()
total_de_votos = total_de_votos.to_frame("total_de_votos")
total_de_votos

,total_de_votos
356,329
318,317
296,307
593,279
2571,278
...,...
5986,1
100304,1
34800,1
83976,1


Notamos que los indices son los "peliculaId" y que la cantidad de filas(9724) es igual al del dataframe peliculas. Con esto podemos agregar el total de votos en una columna nueva en el dataframe peliculas. 

In [13]:
# agregamos total_de_votos en una nueva columna en peliculas
peliculas['total_de_votos'] = total_de_votos
peliculas

,titulo,generos,total_de_votos
peliculaId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0
...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,1.0
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,1.0
193585,Flint (2017),Drama,1.0


Con esto ya podemos ordenar de manera descendente, para tener el top 10 de peliculas mas vistas.

In [20]:
# pedimos que primero las ordene y luego que nos muestre las N primeras
N = 10 # se puede cambiar
peliculas.sort_values("total_de_votos", ascending = False).head(N)

,titulo,generos,total_de_votos
peliculaId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0
110,Braveheart (1995),Action|Drama|War,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0


Con esto ya tenemos una lista con peliculas a recomendar, mision cumplida!!

# 2era Heurística de Recomendacion: Ranking de mejor evaluadas
Que una pelicula sea popular no nos dice si es buena o no, tal vez haya alguna que sea popular pero que sus calificaciones promedio sean bajas o al reves. 

Entonces en este caso vamos a promediar las calificaciones de cada pelicula y lo agregaremos al dataset peliculas, al igual que con la 1era heurística.

In [29]:
# agrupamos los datos por peliculaId luego se promedian y 
# nos quedamos solo con la columna nota
nota_promedio = notas.groupby("peliculaId") #<- se agrupan los datos por peliculaId
nota_promedio = nota_promedio.mean() #<- promediamos estos grupos
nota_promedio = nota_promedio["nota"] #<- nos quedamos solo con la columna nota
nota_promedio = nota_promedio.to_frame("nota_promedio")
nota_promedio

,nota_promedio
peliculaId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429
...,...
193581,4.000000
193583,3.500000
193585,3.500000


Ahora tenemos las calificaciones promedios para cada pelicula y podemos agregarlo al dataset pelicula.

Pero antes, notamos que la celda anterior podia haberse reducido a dos lineas, como se muestra a continuacion.

In [30]:
# es el mismo procedimiento solo que lo hacemos todo a la vez
nota_promedio = notas.groupby("peliculaId").mean()["nota"].to_frame("nota_promedio")
nota_promedio

,nota_promedio
peliculaId,
1,3.920930
2,3.431818
3,3.259615
4,2.357143
5,3.071429
...,...
193581,4.000000
193583,3.500000
193585,3.500000


Ahora si, agregamos la nota promedio al dataset.

In [31]:
# agregamos total_de_votos en una nueva columna en peliculas
peliculas['nota_promedio'] = nota_promedio
peliculas

,titulo,generos,total_de_votos,nota_promedio
peliculaId,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818
3,Grumpier Old Men (1995),Comedy|Romance,52.0,3.259615
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0,2.357143
5,Father of the Bride Part II (1995),Comedy,49.0,3.071429
...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,1.0,4.000000
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,1.0,3.500000
193585,Flint (2017),Drama,1.0,3.500000


Con esto ya podemos ordenar de manera descendente, para tener el top 10 de peliculas mejor evaluadas.

In [32]:
# pedimos que primero las ordene y luego que nos muestre las N primeras
N = 10 # se puede cambiar
peliculas.sort_values("nota_promedio", ascending = False).head(N)

,titulo,generos,total_de_votos,nota_promedio
peliculaId,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


Lo logramos, ahora tenemos una lista para recomendar segun la nota promedio...

Espera un momento, ¿esta recomendacion de verdad estara bien?, es decir tienen buenas calificaciones, pero solo una persona las ha visto. Tal vez, sea buena idea el decirle al recomendador que solo nos recomiende por calificacion a las que tengan mas de 100 votos.

In [34]:
# primero le decimos que solo tome las filas que cumplen que en la columna 
# total_de_votos se tengan mas de los votos necesarios para hacer valida la 
# nota_promedio, luego pedimos que las ordene y luego que nos muestre las N primeras
N = 10 # se puede cambiar
votos_necesarios = 100 # son los votos con los que decidimos que una nota promedio es util
peliculas[peliculas["total_de_votos"]>votos_necesarios].sort_values("nota_promedio", ascending = False).head(N)

,titulo,generos,total_de_votos,nota_promedio
peliculaId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000
58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149.0,4.238255
50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,204.0,4.237745
1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance,142.0,4.232394


Ahora si, tenemos una lista con recomendaciones por calificaciones que nos da resultados mas confiables.

# Continuara...
Pero igual puedes intentar inventar una 3era heuristica de recomendacion, no es necesario implementarla, puede ser una combinacion de el total de votos junto a la nota promedio, o tal vez incluir la desviacion estandar de las notas y dividir la nota promedio por las desviacion estandar, asi favoreciendo a las peliculas en las que las notas estan mas juntas. O tal vez otra cosa, cualquier metodo puede ser una posible solucion.